# Analytical application

This notebook shows how the proposed method can be tested using analytical functions found in literature [1] combined with the classic Sellar problem to benchmark MDAO architectures. Various formulations are provided there, the user has to selectively run the cells of each formulation as desired. The problems include an expensive function and a close cheap approximation that lacks local details. 

We start by importing the necessary libraries.


---
[1] Bryson, D. E., and Rumpfkeil, M. P., “Multifidelity Quasi-Newton Method for Design Optimization,”AIAA Journal,Vol. 56, No. 10, 2018, pp. 4074–4086. https://doi.org/10.2514/1.j056840.

In [ ]:
from __future__ import print_function

import numpy as np

from openmdao.api import Component, Group, IndepVarComp, ExecComp, NLGaussSeidel, ScipyGMRES, view_model, SqliteRecorder

Next, we declare the high and low fidelity versions of the main discipline. Only one of the next 5 code cells should be executed at a time.

## McCormick - Sellar proposal


\begin{equation}
    \begin{matrix}
    Minimize \: x_1^2+z_2+y_1+e^{-y_2} \\
    with \: respect\: to \: z_1,z_2,x_1 \\
    subject\: to  \\
    \frac{y_1}{3.16}-1\geq 0  \\
    1-\frac{y_2}{24}\geq 0 \\
    -10\leq z_1 \leq 10 \\
    0\leq z_2 \leq 10 \\
    0\leq x_1 \leq 10 \\
    \\
    Disc. \: 1: \: \left\{\begin{matrix} 
     y_{1_h}=\sin{(z_1+z_2)}+(z_1-z_2)^2-\frac{3}{2}z_1+\frac{5}{2}z_2-0.2y_2+x_1+1 \\
    \\
    y_{1_l}=(z_1-z_2)^2+2z_1+2z_2-0.2y_2+x_1
    \end{matrix} \right. \\
    \\
    Disc. \: 2: \:\: y_2=\sqrt{y_1}+z_1+z_2\\
    \end{matrix}
\end{equation}


In [ ]:
class Dis1High(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to ADFlow component."""
    def __init__(self):
        super(Dis1High, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = z1**2 + z2 + x1 - 0.2*y2"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('High fidelity evaluation will take place with y2= ', params['y2'])                                                                         
        unknowns['y1'] = np.sin(z1 + z2) + (z1 - z2)**2 - 3/2*z1 + 5/2*z2 - 0.2*y2 + x1 + 1
        print('High fidelity evaluation carried out y1= ', unknowns['y1'])
        
class Dis1Low(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to the Panair component in Aerostructures"""
    def __init__(self):
        super(Dis1Low, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = (z1-z2)**2 + 2*z1 + 2*z2 -0.2*y2 + x1"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('Low fidelity evaluation will take place with y2= ', params['y2'])                                                                        
        unknowns['y1'] = (z1-z2)**2 + 2*z1 + 2*z2 -0.2*y2 + x1
        print('Low fidelity evaluation carried out y1= ', unknowns['y1'])

## Brown - Sellar proposal

\begin{equation}
    \begin{matrix}
    Minimize \: x_1^2+z_2+y_1+e^{-y_2} \\
    with \: respect \: to \: z_1,z_2,x_1 \\
    subject \:to  \\
    \frac{y_1}{3.16}-1\geq 0  \\
    1-\frac{y_2}{24}\geq 0 \\
    -1\leq z_1 \leq 2 \\
    -1\leq z_2 \leq 2 \\
    0\leq x_1 \leq 10 \\
    \\
    Disc. \: 1: \: \left\{\begin{matrix} 
     y_{1_h}=(z_1^2)^{z_2^2+1}+(z_2^2)^{z_1^2+1}+x_1-0.2y_2 \\
    \\
    y_{1_l}=10^{-3}+\exp{(z_1^2+z_2^2)}+x_1-0.2y_2
    \end{matrix} \right. \\
    \\
    Disc. \: 2: \:\: y_2=\sqrt{y_1}+z_1+z_2\\
    \end{matrix}
\end{equation}

In [ ]:
class Dis1High(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to ADFlow component."""
    def __init__(self):
        super(Dis1High, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = (z1**2)**(z2**2+1) + (z2**2)**(z1**2+1) - 0.2*y2 + x1"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('High fidelity evaluation will take place with y2= ', params['y2'])                                                                         
        unknowns['y1'] = (z1**2)**(z2**2+1) + (z2**2)**(z1**2+1) - 0.2*y2 + x1
        print('High fidelity evaluation carried out y1= ', unknowns['y1'])
        
class Dis1Low(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to the Panair component in Aerostructures"""
    def __init__(self):
        super(Dis1Low, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = 10**-3+exp(z1**2+z2**2) -0.2*y2 + x1"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('Low fidelity evaluation will take place with y2= ', params['y2'])                                                                        
        unknowns['y1'] = 10**-3+np.exp(z1**2+z2**2) -0.2*y2 + x1
        print('Low fidelity evaluation carried out y1= ', unknowns['y1'])

## Styblinski-Tang  -  Sellar proposal

\begin{equation}
    \begin{matrix}
    Minimize \: x_1^2+z_2+y_1+e^{-y_2} \\
    with \: respect \: to \: z_1,z_2,x_1 \\
    subject \: to  \\
    \frac{y_1}{3.16}-1\geq 0  \\
    1-\frac{y_2}{24}\geq 0 \\
    -5\leq z_1 \leq 5 \\
    -5\leq z_2 \leq 5 \\
    0\leq x_1 \leq 10 \\
    \\
    Disc. \: 1: \: \left\{\begin{matrix} 
     y_{1_h}=\frac{1}{2}\left[\left(z_1^4-16z_1^2+5z_1\right) \right. \left.+\left(z_2^4-16z_2^2+5z_2 \right) \right]+x_1-0.2y_2 \\
    \\
    y_{1_l}=z_1^4\cos\left(\frac{2\pi z_1}{5}\right)+z_2^4\cos\left(\frac{2\pi z_2}{5}\right) +x_1-0.2y_2
    \end{matrix} \right. \\
    \\
    Disc.\: 2: \:\: y_2=\sqrt{y_1}+z_1+z_2\\
    \end{matrix}
\end{equation}

In [ ]:
class Dis1High(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to ADFlow component."""
    def __init__(self):
        super(Dis1High, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = 1/2*((z1**4 - 16*z1**2 + 5*z1)+(z2**4 - 16*z2**2 + 5*z2) - 0.2*y2 + x1)"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('High fidelity evaluation will take place with y2= ', params['y2'])
        unknowns['y1'] = 1/2*((z1**4 - 16*z1**2 + 5*z1)+(z2**4 - 16*z2**2 + 5*z2)) - 0.2*y2 + x1 
        print('High fidelity evaluation carried out y1= ', unknowns['y1'])

class Dis1Low(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to the Panair component in Aerostructures"""
    def __init__(self):
        super(Dis1Low, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = z1**4*np.cos(2*np.pi*z1/5) + z2**4*np.cos(2*np.pi*z2/5 - 0.2*y2 + x1"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('Low fidelity evaluation will take place with y2= ', params['y2'])                                                                        
        unknowns['y1'] = z1**4*np.cos(2*np.pi*z1/5) + z2**4*np.cos(2*np.pi*z2/5) - 0.2*y2 + x1
        print('Low fidelity evaluation carried out y1= ', unknowns['y1'])

## Rosenbrock - Sellar proposal

\begin{equation}
    \begin{matrix}
    Minimize \: x_1^2+z_2+y_1+e^{-y_2} \\
    with\:respect\:to \: z_1,z_2,x_1 \\
    subject\:to  \\
    \frac{y_1}{3.16}-1\geq 0  \\
    1-\frac{y_2}{24}\geq 0 \\
    -2\leq z_1 \leq 2 \\
    -2\leq z_2 \leq 2 \\
    0\leq x_1 \leq 10 \\
    \\
    Disc.\: 1: \: \left\{\begin{matrix} 
     y_{1_h}=(1-z_1)^2+100\left(z_2-z_1^2\right)^2+x_1-0.2y_2 
    \\
    y_{1_l}=1+50z_1^2\left[(z_2-2)^2+2\right]+x_1-0.2y_2
    \end{matrix} \right. \\
    \\
    Disc.\: 2: \:\: y_2=\sqrt{y_1}+z_1+z_2\\
    \end{matrix}
\end{equation}

In [ ]:
class Dis1High(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to ADFlow component."""
    def __init__(self):
        super(Dis1High, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = (1-z1)**2 + 100*(z2-z1**2)**2 - 0.2*y2 + x1 """

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('High fidelity evaluation will take place with y2= ', params['y2'])                                                                         
        unknowns['y1'] = (1-z1)**2 + 100*(z2-z1**2)**2 - 0.2*y2 + x1 
        print('High fidelity evaluation carried out y1= ', unknowns['y1'])
        
class Dis1Low(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to the Panair component in Aerostructures"""
    def __init__(self):
        super(Dis1Low, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = 1 + 50*z1**2*((z2-2)**2 + 2) -0.2*y2 + x1"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('Low fidelity evaluation will take place with y2= ', params['y2'])                                                                        
        unknowns['y1'] = 1 + 50*z1**2*((z2-2)**2 + 2) -0.2*y2 + x1
        print('Low fidelity evaluation carried out y1= ', unknowns['y1'])

## Trigonometric - Sellar proposal

\begin{equation}
    \begin{matrix}
    Minimize \: x_1^2+z_2+y_1+e^{-y_2} \\
    with \:respect\: to \: z_1,z_2,x_1 \\
    subject to  \\
    \frac{y_1}{3.16}-1\geq 0  \\
    1-\frac{y_2}{24}\geq 0 \\
    -1.5\leq z_1 \leq 3 \\
    -1.5\leq z_2 \leq 3 \\
    0\leq x_1 \leq 10 \\
    \\
    Disc. \:1: \: \left\{\begin{matrix} 
     y_{1_h}=\sum_{i=1}^{2}\left[2-\sum_{j=1}^{2}\cos(x_j)  +i(1-\cos(x_i)-\sin(x_i))\right]^2 
    \\
    y_{1_l}=10^{-3}+\sum_{i=1}^{2} 5\left(x_i-\frac{1}{4}\right)-0.2y_2+x_1
    \end{matrix} \right. \\
    \\
    Disc.\: 2: \:\: y_2=\sqrt{y_1}+z_1+z_2\\
    \end{matrix}
\end{equation}

In [ ]:
class Dis1High(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to ADFlow component."""
    def __init__(self):
        super(Dis1High, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = 1/2*((z1**4 - 16*z1**2 + 5*z1)+(z2**4 - 16*z2**2 + 5*z2) - 0.2*y2 + x1)"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('High fidelity evaluation will take place with y2= ', params['y2'])
        unknowns['y1'] = (2 - (np.cos(z1) + (1 - np.cos(z1) - np.sin(z1))) - (np.cos(z2) + (1 - np.cos(z1) - np.sin(z1))))**2 + \
        (2 - (np.cos(z1) + 2*(1 - np.cos(z2) - np.sin(z2))) - (np.cos(z2) + 2*(1 - np.cos(z2) - np.sin(z2))))**2 - 0.2*y2 + x1
        print('High fidelity evaluation carried out y1= ', unknowns['y1'])

class Dis1Low(Component):
    """Component containing Discipline 1 -- High Fidelity."""
    """Equivalent to the Panair component in Aerostructures"""
    def __init__(self):
        super(Dis1Low, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Local Design Variable
        self.add_param('x', val=0.)

        # Coupling parameter
        self.add_param('y2', val=0.)

        # Coupling output
        self.add_output('y1', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y1 = 10**-3 + 5*(z1 - 1/4)**2 + 5*(z2 - 1/4)**2 - 0.2*y2 + x1"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        x1 = params['x']
        y2 = params['y2']

        print('Low fidelity evaluation will take place with y2= ', params['y2'])
        unknowns['y1'] = 10**-3 + 5*(z1 - 1/4)**2 + 5*(z2 - 1/4)**2 - 0.2*y2 + x1
        print('Low fidelity evaluation carried out y1= ', unknowns['y1'])

We now define discipline 2, which is the same for all formulations.

In [ ]:
class Dis2(Component):
    """Component containing Discipline 2."""
    """Equivalent to the NastranStatic component in aerostructures"""
    def __init__(self):
        super(Dis2, self).__init__()

        # Global Design Variable
        self.add_param('z', val=np.zeros(2))

        # Coupling parameter
        self.add_param('y1', val=0.)

        # Coupling output
        self.add_output('y2', val=1.0)

    def solve_nonlinear(self, params, unknowns, resids):
        """Evaluates the equation
        y2 = y1**(.5) + z1 + z2"""

        z1 = params['z'][0]
        z2 = params['z'][1]
        y1 = params['y1']

        # Note: this may cause some issues. However, y1 is constrained to be
        # above 3.16, so lets just let it converge, and the optimizer will
        # throw it out
        y1 = abs(y1)

        print('Discipline 2 evaluation will be carried out with y1= ', params['y1'])
        unknowns['y2'] = y1**.5 + z1 + z2
        print('Discipline 2 evaluation carried out y2= ', unknowns['y2'])

As in the full case, we need to define a "filter" discipline that exchanges data between fidelities.

In [ ]:
class Filter(Component):
    """Component to coordinate data transfer between fidelities"""
    """Equivalent to the Filter component in aerostructures"""
    def __init__(self, fidelity):
        super(Filter, self).__init__()
                
        #Displacement field from the structures component
        self.add_param('y2', val=0.0)

        #Displacement field from the Lo-Fi component
        self.add_param('y2_l', val=0.0)

        #Displacements after decision
        self.add_output('y2_s', val=0.0)
        
        #Aux variables
        self.y2_l_old = 0
        
        #Fidelity level
        self.fidelity = fidelity
        
        if self.fidelity == 'high':
            self.aux = 1
        else:
            self.aux = 0
    def solve_nonlinear(self, params, unknowns, resids):
        """Selects the appropriate start value"""
        y2 = params['y2']
        y2_l = params['y2_l']
                
        #Changes the value of aux when a new iteration of the optimizer started
        print('y2 old = ', self.y2_l_old)
        print('y2 =     ', y2_l)
        print('aux', self.aux)
        if self.aux == 1 and np.array_equal(self.y2_l_old, y2_l) == False: 
            if self.fidelity == 'high':
                self.aux = 1
            else:
                self.aux = 0
                
        #Changes fidelities if needed
        if self.aux == 0:
            unknowns['y2_s'] = y2_l
            self.y2_l_old = np.copy(y2_l)  
            self.aux = 1
            print('::::::::Fidelity changed, displacement field imported::::::::')
            
        #Keeps current displacement field during the same MDA loop
        else:
            unknowns['y2_s'] = y2
            print('::::::::Displacement field kept from previous iteration::::::::')

Next, we create the MDA groups that will be used on the main script. 

In [ ]:
class MDA_l(Group):
    """ Group containing the Sellar MDA. This version uses the disciplines
    without derivatives."""

    def __init__(self):
        super(MDA_l, self).__init__()

        self.add('d1_l', Dis1Low(), promotes=['x', 'z'])
        self.add('d2_l', Dis2(), promotes=['z'])
        self.add('filter_l', Filter(fidelity))                                      

        self.nl_solver = NLGaussSeidel()
        self.nl_solver.options['atol'] = 1.0e-5
        self.ln_solver = ScipyGMRES()

class MDA_h(Group):
    """ Group containing the Sellar MDA. This version uses the disciplines
    without derivatives."""

    def __init__(self):
        super(MDA_h, self).__init__()

        self.add('d1_h', Dis1High(), promotes=['x', 'z'])
        self.add('d2_h', Dis2(), promotes=['z'])
        self.add('filter', Filter(fidelity))

        self.nl_solver = NLGaussSeidel()
        self.nl_solver.options['atol'] = 1.0e-10
        self.ln_solver = ScipyGMRES()

Now we set up the root problem and run it to see the results.

In [ ]:
if __name__ == '__main__':

    from openmdao.api import Problem, ScipyOptimizer, SqliteRecorder

    #Initialize auxiliary variables
    y2 = 0
    y2_l = 0
    fidelity = 'multi'
    
    #Set the casename 
    casename = 'McCormick' # THE USER MUST CHANGE THIS VARIABLE ACCORDING TO THE CASE IMPORTANT FOR POST-PROCESSING
    # Setup and run the model.

    top = Problem()
    top.root = root = Group()
    root.ln_solver = ScipyGMRES()
    #Add independet variables
    root.add('px', IndepVarComp('x', 1.0), promotes=['x'])
    root.add('pz', IndepVarComp('z', np.array([5.0, 2.0])), promotes=['z'])
    
    root.add('mda_group_l', MDA_l(), promotes=['*'])
    root.mda_group_l.connect('d1_l.y1','d2_l.y1')
    root.mda_group_l.connect('d2_l.y2','filter_l.y2')
    root.mda_group_l.connect('filter_l.y2_s','d1_l.y2')                                                   
    root.add('mda_group_h', MDA_h(), promotes=['*'])
    root.mda_group_h.connect('d1_h.y1','d2_h.y1')
    root.mda_group_h.connect('d2_h.y2','filter.y2')
    root.mda_group_h.connect('filter.y2_s','d1_h.y2')
    
    
    root.add('obj_cmp', ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)', z=np.array([0.0, 0.0]) ), promotes=['obj', 'x', 'z', 'y1', 'y2'])
    root.connect('d1_h.y1','y1')
    root.connect('d2_h.y2','y2')
    root.add('con_cmp1', ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
    root.connect('d1_h.y1','y1')
    root.add('con_cmp2', ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])
    root.connect('d2_h.y2','y2')
    root.connect('d2_h.y2','filter_l.y2_l')                                       
    root.connect('d2_l.y2','filter.y2_l')

    top.driver = ScipyOptimizer()
    top.driver.options['optimizer'] = 'COBYLA'
    top.driver.options['tol'] = 1.0e-10

    top.driver.add_desvar('z', lower=np.array([-5.0, -5.0]), upper=np.array([5.0, 5.0]))
    #Add constraints manually, COBYLA doesn't respect upper and lower bounds
    root.add('z1_lower_bound', ExecComp('z1_l = z[0]', z=np.array([0.0, 0.0])), promotes=['*'])
    root.add('z1_upper_bound', ExecComp('z1_u = z[0]', z=np.array([0.0, 0.0])), promotes=['*'])
    top.driver.add_constraint('z1_l', lower=-5)
    top.driver.add_constraint('z1_u', upper=5)
    root.add('z2_lower_bound', ExecComp('z2_l = z[1]', z=np.array([0.0, 0.0])), promotes=['*'])
    root.add('z2_upper_bound', ExecComp('z2_u = z[1]', z=np.array([0.0, 0.0])), promotes=['*'])
    top.driver.add_constraint('z2_l', lower=-5)
    top.driver.add_constraint('z2_u', upper=5)
    top.driver.add_desvar('x', lower=0.0, upper=10.0)
    root.add('x_lower_bound', ExecComp('x_l = x'), promotes=['*'])
    root.add('x_upper_bound', ExecComp('x_u = x'), promotes=['*'])
    top.driver.add_constraint('x_l', lower=0)
    top.driver.add_constraint('x_u', upper=10)
    top.driver.add_objective('obj')
    top.driver.add_constraint('con1', upper=0.0)
    top.driver.add_constraint('con2', upper=0.0)
    rec = SqliteRecorder(casename + '.sqlite3')
    rec.options['includes'] = ['obj', 'con1', 'con2']
    rec.options['record_derivs'] = True
    top.driver.add_recorder(rec)

    root.mda_group_l.set_order(['filter_l', 'd1_l', 'd2_l'])
    root.mda_group_h.set_order(['filter', 'd1_h', 'd2_h'])
    root.set_order(['pz','z1_lower_bound','z1_upper_bound','z2_lower_bound','z2_upper_bound','px','mda_group_l','mda_group_h','con_cmp1','con_cmp2','obj_cmp','x_lower_bound','x_upper_bound'])                                                                                                                                                                                                                                                  
    
    #Recorder Lo-Fi
    recorder_l = SqliteRecorder('mda_l.sqlite3')
    recorder_l.options['record_metadata'] = False
    #Recorder Hi-Fi
    recorder_h = SqliteRecorder('mda_h.sqlite3')
    recorder_h.options['record_metadata'] = False
    # recorder.options['includes'] =
    top.root.mda_group_l.nl_solver.add_recorder(recorder_l)
    top.root.mda_group_h.nl_solver.add_recorder(recorder_h)
    
    top.setup()
    view_model(top, show_browser=False)
    top.run()

    print("\n")
    print( "Minimum found at (%f, %f, %f)" % (top['z'][0],
                                             top['z'][1],
                                             top['x']))
    print("Coupling vars: %f, %f" % (top['y1'], top['y2']))
    print("Minimum objective: ", top['obj'])

## Postprocessing
The script below generates 3 plots that show the evolution of the constraints and the objective function with respect to the iterations. 

In [ ]:
%run ./optim_post.py #Check the pdf files created on the root folder